# 元编程（Meta-Programming）

什么是元编程呢？

简而言之，代码是可以操作的对象，我可以通过元编程来**自动生成代码， 并执行代码**

## 表达式

### 通过字符串创建表达式

In [1]:
prog = "1+1"

"1+1"

In [2]:
ex1 = Meta.parse(prog)

:(1 + 1)

In [3]:
eval(ex1)

2

In [4]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


### 通过`Expr`手动创建

In [5]:
Expr(:call, :+, 1, 1)

:(1 + 1)

### 通过`quote`创建多个表达式

In [6]:
ex3 = quote
    x = 1
    y = 2
end

quote
    #= In[6]:2 =#
    x = 1
    #= In[6]:3 =#
    y = 2
end

In [7]:
dump(ex3)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[6]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[6]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 2


可以看出`quote`通过`array`嵌套了多个表达式， 他的`head`是`:block`（中间有两行是注释）

In [8]:
esc(ex3)

:($(Expr(:escape, quote
    #= In[6]:2 =#
    x = 1
    #= In[6]:3 =#
    y = 2
end)))

### 直接通过`:()`创建

In [9]:
:(1+1) == Expr(:call, :+, 1, 1)

true

In [10]:
ex2 = :(2 + 3 * a + b) 

:(2 + 3a + b)

In [11]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 3
        3: Symbol a
    4: Symbol b


julia的表达式通常尽可以包含`符号、子表达式、字面值`， 上式中`+, b`是符号， `:(3 * a)`是子表达式， `2`是字面值，上式是一个嵌套的表达式， **`a, b`的值还不知道， 在这里被当做符号对待**

<font style="color:purple;font-size:14pt;">如果`a, b`已知， 那么通过`$`进行插值， 可以使构造的表达式中的`a, b`替换为变量`a, b`的实际值</font>

In [12]:
a = 1
b = 1
:(2 + 3 * a + b) # no $

:(2 + 3a + b)

In [13]:
# 仍然是可以求值的
eval(:(2 + 3 * a + b))

6

In [14]:
:(2 + 3 * $a + $b) #！！ with $ 

:(2 + 3 * 1 + 1)

In [15]:
# 很自然的可以求值
eval(:(2 + 3 * $a + $b))

6

In [16]:
# splatting interploration

args = [:x, :y, :z]
:(f(1, $(args...)))

:(f(1, x, y, z))

可以看出来整个工作的流程就是， 利用`Expr`表达式生成代码， 然后使用`eval`函数求值， 达到自动生成代码， 运行代码的目的

## 宏

宏像是一个函数， 但是又与函数不太相同， **宏会返回一个Expr表达式， 这个表达式不经过`eval`直接自动运行**, `@macroexpand`可以看到生成的表达式。

为什么需要宏呢？ 宏的内部可以执行其它的函数， 最后再去构造和执行Expr表达式

### 定义宏

In [41]:
macro sayhello()
    return :(println("Hello World!"))
end

@sayhello (macro with 2 methods)

In [60]:
@sayhello

Hello World!


In [61]:
macro sayhello0(name)
    return :(println("Hello World! ", name))
end
macro sayhello1(name)
    return :(println("Hello World! ", $name))
end

@sayhello1 (macro with 1 method)

In [62]:
@macroexpand @sayhello1 "XJZ"

:(Main.println("Hello World! ", "XJZ"))

In [63]:
@macroexpand @sayhello0 "XJZ"

:(Main.println("Hello World! ", Main.name))

???

注意`sayhello(name)`这个宏中用到了插值`$name`, 明明宏提供了`name`这个位置参数， 为啥呢？ 

因为我们返回的是一个表达式， 返回值是一个构造`Expr`的过程， 在构造`Expr`的过程中，使用任何变量都要进行插值，手动狗头

### 理解宏的运行

In [23]:
macro twostep(arg...)
    println("I execute at parse time. The argument is: ", arg...)
    return :(println("I execute at runtime. The argument is: ", $(arg...)))
end

@twostep (macro with 1 method)

In [24]:
@macroexpand @twostep(1, 2, 3)

I execute at parse time. The argument is: 123


:(Main.println("I execute at runtime. The argument is: ", 1, 2, 3))

可以看出， 仅生成表达式时， 会执行中间定义的命令，即**生成表达式也属于执行命令**

执行宏命令时， 会运行宏内部的的所有语句命令， 以及"`eval`"该宏返回的表达式

In [64]:
@twostep 1, 2, 3

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


### 卫生宏

卫生宏，英文为`Hygienic Macro`， 指宏内定义的变量与宏运行环境中的变量不冲突， Julia默认帮你做到了这一点(默认定义的变量为`local`)， 但是有些情况我们希望宏能够修改运行环境中的一些变量， 那么就可以使用`global`，声明该变量为全局变量， 这样可以修改全局作用域中的变量。

然而存在一种情况， 例如函数内使用了宏， 这个宏想修改函数内的变量的值而不是全局作用域的值， 这样怎么搞？， 答`esc()`

In [205]:
# 局部变量， 运行环境内的变量没有任何影响
# 默认quote内定义的为局部变量
x = 0.1
y = 0.2
macro envtest1()
    return quote
         x = 1 # 前边隐藏了local
         y = 2 # 前边隐藏了local
    end
end
@envtest1, x, y

(2, 0.1, 0.2)

In [206]:
# 可以看出Julia对局部变量进行了重命名保证唯一性， 这样局quote中定义的变量就不会与全局变量冲突了
@macroexpand @envtest1

quote
    #= In[205]:7 =#
    var"#243#x" = 1
    #= In[205]:8 =#
    var"#244#y" = 2
end

In [207]:
# 另一种写法， 修改全局变量x的值， y为局部变量
x = 0.1
y = 0.2
macro envtest2()
    expr = quote
         global x = 1
         y = 2
    end
    return expr
end
@envtest2, x, y

(2, 1, 0.2)

In [208]:
# 仅仅y被重命名了
@macroexpand @envtest2

quote
    #= In[207]:6 =#
    global x = 1
    #= In[207]:7 =#
    var"#246#y" = 2
end

In [212]:
# 以纳秒（ns）的形式的获取当前时间， 内置函数
time_ns()
macro envtest3(ex)
    return quote
        t1 = time_ns()
        $(ex)
        t2 = time_ns()
        elapsed = (t2 - t1)/1e9
        println("elapsed time is ", elapsed)
        println(t1, "\n", t2)
    end
end

@envtest3 (macro with 1 method)

In [215]:
@macroexpand @envtest3 z = 1 + 2 # 这里的z竟然也是局部变量， 相当于把z加入了quote中， 也是在quote中定义为局部变量

quote
    #= In[212]:5 =#
    var"#257#t1" = Main.time_ns()
    #= In[212]:6 =#
    var"#258#z" = 1 + 2
    #= In[212]:7 =#
    var"#259#t2" = Main.time_ns()
    #= In[212]:8 =#
    var"#260#elapsed" = (var"#259#t2" - var"#257#t1") / 1.0e9
    #= In[212]:9 =#
    Main.println("elapsed time is ", var"#260#elapsed")
    #= In[212]:10 =#
    Main.println(var"#257#t1", "\n", var"#259#t2")
end

In [216]:
function test_env()
    t1 = 1
    t2 = 2
    return t2 + t2
end

test_env (generic function with 1 method)

In [217]:
@macroexpand @envtest3 test_env()

quote
    #= In[212]:5 =#
    var"#261#t1" = Main.time_ns()
    #= In[212]:6 =#
    Main.test_env()
    #= In[212]:7 =#
    var"#262#t2" = Main.time_ns()
    #= In[212]:8 =#
    var"#263#elapsed" = (var"#262#t2" - var"#261#t1") / 1.0e9
    #= In[212]:9 =#
    Main.println("elapsed time is ", var"#263#elapsed")
    #= In[212]:10 =#
    Main.println(var"#261#t1", "\n", var"#262#t2")
end

看以上宏展开结果， quote中所有宏调用的函数都是`Main.function`，即全局作用域中的函数

In [218]:
macro envtest3(ex)
    return quote
        t1 = time_ns()
        $(esc(ex))
        t2 = time_ns()
        elapsed = (t2 - t1)/1e9
        println("elapsed time is ", elapsed)
        println(t1, "\n", t2)
    end
end

@envtest3 (macro with 1 method)

In [219]:
@macroexpand @envtest3 test_env()

quote
    #= In[218]:3 =#
    var"#264#t1" = Main.time_ns()
    #= In[218]:4 =#
    test_env()
    #= In[218]:5 =#
    var"#265#t2" = Main.time_ns()
    #= In[218]:6 =#
    var"#266#elapsed" = (var"#265#t2" - var"#264#t1") / 1.0e9
    #= In[218]:7 =#
    Main.println("elapsed time is ", var"#266#elapsed")
    #= In[218]:8 =#
    Main.println(var"#264#t1", "\n", var"#265#t2")
end

加入`esc`以后， `Main.`被去掉了， 即**当前作用域能看到的`test_env()`我都可以去调用**

In [232]:
# 案例
xx = 100
macro zeroxx()
    return esc(:(xx = 0))
end
macro zeroxx2()
    return quote
        global xx = 88
    end
end

@zeroxx2 (macro with 1 method)

In [233]:
function foo()
    xx = 99
    @zeroxx
    return xx
end
function baz()
    xx = 99
    @zeroxx2
    return xx
end

baz (generic function with 1 method)

In [234]:
foo(), xx

(0, 100)

In [235]:
baz(), xx

(88, 88)

这可案例可以清楚的看到， `global`声明会直接修改全局作用域的变量， 这样其实不太好， 而`esc()`仅会修改上层作用域的变量， 毕竟没有`Main.`, 宏出现在哪个作用域里， 他就会调用哪个作用域里都可以看到的函数

### 案例一：assert

In [268]:
macro assert(ex)
    return :($ex ? nothing : AssertionError("not equal"))
end

@assert (macro with 1 method)

In [269]:
@assert 1 == 12

AssertionError("not equal")

In [270]:
@macroexpand @assert 1 == 2.0

:(if 1 == 2.0
      Main.nothing
  else
      Main.AssertionError("not equal")
  end)

### 案例二：Fibonacci数列

$$a_0 = 1$$
$$a_1 = 1$$
$$a_i = a_{i-1} + a_{i-2}, \text{if}\ i \geq 2$$

**（1）传统思路：查表**

In [79]:
function fib1(n)
    rst = zeros(n)
    if n == 1
        return 1.0
    elseif n == 2
        return 1.0
    elseif n > 2 
        rst[1:2] .= 1
        for i in 3:length(rst)
            rst[i] = rst[i - 1] + rst[i - 2]
        end
        return rst[end]
    end 
end

fib1 (generic function with 1 method)

In [80]:
using BenchmarkTools

In [81]:
@btime fib1(10)

  36.455 ns (1 allocation: 144 bytes)


55.0

以上实现了一个可以查表的fibnacci数列的计算方面，避免了递归， 运算性能还不错， 几乎是线性复杂度， 他的原理是**程序在运行时创建一个向量， 存储之前的计算结果， 后续计算可以利用之前的计算结果**

**（2）编译期查表**

### `@generated`

`@generated`可以定义一种函数的生成方式， 我们仅能在函数体内获得参数的类型， 无法获得参数的值

In [276]:
@generated function gpp(x, y)
   if (x <: Number) & (y <: Number)
        Core.println(x, y)
        return :(x + y)
    else
        @error "wrong type"
    end
end

gpp (generic function with 1 method)

In [277]:
gpp(1, 2)

Int64Int64


3